# Argentina programa 4.0 - Módulo 3: Aprendizaje Automático
---
# Ejercicios semana 7 - CNNs


## Fashion-MINST


Al igual que la semana anterior, usen el conjunto de datos fashion-MINST. trayendo el el dataset desde `keras.datasets.fashion_mnist`:

1.   Separen en train, validation y test (hecho...)
2.   Normalicen de la misma forma que hicimos con CIFAR-10 o MNIST
3.   Hagan una CNN simple para este problema (clasificar 10 clases): una sola capa convolucional con 32 filtros y kernel de 3x3, un maxpooling de 2x2 y la red totalmente conectada del final de 256 unidades.
4.   Usen un early stopping monitoreando la loss en el conjunto de validación con una paciencia de 10 épocas, entrenen hasta que pare y anoten el resultado de la accuracy en el conjunto de validación. **Este es su modelo de base**.
5.   Ahora busquen una nueva arquitectura que le gane a este modelo. Prueben ir agregando de a una capa convolucional y ver si mejora, cambiar los tamaños y las cantidades de los filtros, intercalen con pooling, recuerden que tambien pueden usar dropout, jugar con el stride y el padding, etc.

Como recomendación sean sistemáticos, vayan probando de a un cambio a la vez y viendo si mejora, empiecen con las capas convolucionales y de pooling (se suele recomendar que no haya más de dos convolucionales por cada pooling) agregar más no mejore prueben aumentar el número de filtros sobre las capas finales, achicar o agrandar los kernels, aplicar un PCA, etc. Recuerden que pueden usar tensorboard o [weight and biases](https://wandb.ai/wandb_fc/articles/reports/Intro-to-Keras-with-Weights-Biases--Vmlldzo1NDM0ODEz) para comparar las diferentes arquitecturas.

Una vez que tengan un modelo final (o sea, ya se cansaron de probar :P) comparen el rendimiento de esta red contra la de la semana pasada y asi quieren compartan el resultado sobre le conjunto de validación con sus compañeros. La arquitectura ganadora de todos ustedes que sea probada sobre el conjunto de testeo.

In [1]:
#Las estrellas
import tensorflow as tf
from tensorflow import keras

In [2]:
#traemos ya test y train
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

#60k en train 10k en test
X_train_full = X_train_full.reshape((60000, 28, 28, 1))
X_test = X_test.reshape((10000, 28, 28, 1))

#separamos train en validación
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# a partir de ahora ustedes

X_valid = X_valid / 255.
X_train = X_train / 255.
X_test = X_test / 255.


4422102/4422102 [==============================] - 0s 0us/step


La red base del enunciado

In [ ]:
#Red, empezamos simple

input_ = keras.layers.Input(shape=(X_train.shape[1:])) #En este caso es necesario definir el Layer de Inputs
#capas convolucionales
conv1 = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(input_)
#con esto achico a un cuarto los mapas de características
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
#la salida se aplana para pasarsela a una red totalmente conectada
flatten = keras.layers.Flatten()(pool1)
hidden1 = keras.layers.Dense(256, activation="relu")(flatten)
output = keras.layers.Dense(10, activation="softmax")(hidden1)
model = keras.models.Model(inputs=[input_], outputs=[output])

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               1605888   
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                           

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
epochs = 100
#Esto faltó definirlo en el enunciado, uso el defautl
batch_size = 32

history = model.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,
                    validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb])


Epoch 1/100
1719/1719 [==============================] - 10s 4ms/step - loss: 0.7167 - accuracy: 0.7561 - val_loss: 0.5064 - val_accuracy: 0.8256
Epoch 2/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4802 - accuracy: 0.8277 - val_loss: 0.5018 - val_accuracy: 0.8220
Epoch 3/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4288 - accuracy: 0.8484 - val_loss: 0.4181 - val_accuracy: 0.8506
Epoch 4/100
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3944 - accuracy: 0.8596 - val_loss: 0.3937 - val_accuracy: 0.8608
Epoch 5/100
1719/1719 [==============================] - 8s 4ms/step - loss: 0.3678 - accuracy: 0.8700 - val_loss: 0.3504 - val_accuracy: 0.8784
Epoch 6/100
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3476 - accuracy: 0.8751 - val_loss: 0.3316 - val_accuracy: 0.8852
Epoch 7/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3309 - accuracy: 0.8819 - val_loss: 0.3320 - val_a

In [ ]:
accuracy = model.evaluate(X_valid, y_valid)[1]

print("Accuracy en Validación: ", accuracy)

157/157 [==============================] - 1s 6ms/step - loss: 0.2328 - accuracy: 0.9168
Accuracy en Validación:  0.9168000221252441


0.916, a ver que pasa con otra capa

In [ ]:
keras.backend.clear_session()

input_ = keras.layers.Input(shape=(X_train.shape[1:]))
conv1 = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(input_)
conv2 = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(conv1)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

flatten = keras.layers.Flatten()(pool1)
hidden1 = keras.layers.Dense(256, activation="relu")(flatten)
output = keras.layers.Dense(10, activation="softmax")(hidden1)

model_1 = keras.models.Model(inputs=[input_], outputs=[output])

model_1.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


epochs = 100
batch_size = 32

history = model_1.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,
                    validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb])

Epoch 1/100
1719/1719 [==============================] - 10s 6ms/step - loss: 0.6641 - accuracy: 0.7640 - val_loss: 0.4804 - val_accuracy: 0.8242
Epoch 2/100
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4484 - accuracy: 0.8384 - val_loss: 0.3943 - val_accuracy: 0.8612
Epoch 3/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3858 - accuracy: 0.8610 - val_loss: 0.3739 - val_accuracy: 0.8648
Epoch 4/100
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3485 - accuracy: 0.8731 - val_loss: 0.3606 - val_accuracy: 0.8728
Epoch 5/100
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3220 - accuracy: 0.8841 - val_loss: 0.3273 - val_accuracy: 0.8788
Epoch 6/100
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3008 - accuracy: 0.8911 - val_loss: 0.3137 - val_accuracy: 0.8890
Epoch 7/100
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2836 - accuracy: 0.8968 - val_loss: 0.3109 - val

In [ ]:
accuracy = model_1.evaluate(X_valid, y_valid)[1]

print("Accuracy en Validación: ", accuracy)

157/157 [==============================] - 0s 3ms/step - loss: 0.2453 - accuracy: 0.9140
Accuracy en Validación:  0.9139999747276306


No mejoró mucho, de hecho dio un poquito peor (no es significativo el número, pero bueno) meto otro maxpool y de paso cambio SGD por ADAM que en teoría funciona mejor

In [ ]:
keras.backend.clear_session()

input_ = keras.layers.Input(shape=(X_train.shape[1:])) #En este caso es necesario definir el Layer de Inputs

conv1 = keras.layers.Conv2D(filters=16, kernel_size=(5, 5), padding='same', activation='relu')(input_)
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = keras.layers.Conv2D(filters=32, kernel_size=(5, 5), padding='valid', activation='relu')(pool1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

flatten = keras.layers.Flatten()(pool2)

hidden1 = keras.layers.Dense(256, activation="relu")(flatten)
output = keras.layers.Dense(10, activation="softmax")(hidden1)
model_3 = keras.models.Model(inputs=[input_], outputs=[output])

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

epochs = 100
batch_size = 32

model_3.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model_3.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 32)        12832     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 32)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 800)               0     

In [ ]:
history = model_3.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,
                    validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb])

Epoch 1/100
1719/1719 [==============================] - 9s 4ms/step - loss: 0.4655 - accuracy: 0.8325 - val_loss: 0.3181 - val_accuracy: 0.8832
Epoch 2/100
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3040 - accuracy: 0.8898 - val_loss: 0.3041 - val_accuracy: 0.8854
Epoch 3/100
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2587 - accuracy: 0.9036 - val_loss: 0.2592 - val_accuracy: 0.9044
Epoch 4/100
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2254 - accuracy: 0.9175 - val_loss: 0.2523 - val_accuracy: 0.9086
Epoch 5/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.1995 - accuracy: 0.9263 - val_loss: 0.2536 - val_accuracy: 0.9040
Epoch 6/100
1719/1719 [==============================] - 11s 6ms/step - loss: 0.1768 - accuracy: 0.9337 - val_loss: 0.2438 - val_accuracy: 0.9128
Epoch 7/100
1719/1719 [==============================] - 11s 6ms/step - loss: 0.1580 - accuracy: 0.9416 - val_loss: 0.2434 - val

In [ ]:
accuracy = model_3.evaluate(X_valid, y_valid)[1]

print("Accuracy en Validación: ", accuracy)

157/157 [==============================] - 0s 3ms/step - loss: 0.2434 - accuracy: 0.9136
Accuracy en Validación:  0.9136000275611877


De nuevo, nada, solo hay un cambio en la cantidad de épocas en las que llega. Hice varias pruebas cambiando capas convolucionales (principalmente achicando tamaño de kernel por adentro y aumentando filtros), pero no hubo una mejora, incluso varias veces daba peor. Así que empecé a meter mano en las totalmente conectadas, ahí mejoró un toque:

In [3]:
#paso a la api secuencial, me habia hartado de meter errores al poner y sacar capas

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

keras.backend.clear_session()

# Crear un modelo secuencial
model_X = Sequential()

# Definir las capas del modelo
model_X.add(Conv2D(filters=16, kernel_size=(5, 5), padding='same', activation='relu', input_shape=X_train.shape[1:]))
model_X.add(MaxPooling2D(pool_size=(2, 2)))
model_X.add(Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu'))
model_X.add(MaxPooling2D(pool_size=(2, 2)))
model_X.add(Flatten())
model_X.add(Dropout(0.2))
model_X.add(Dense(128, activation='relu'))
model_X.add(Dropout(0.2))
model_X.add(Dense(64, activation='relu'))
model_X.add(Dropout(0.2))
model_X.add(Dense(32, activation='relu'))
model_X.add(Dense(10, activation='softmax'))

# Compilar el modelo
model_X.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

# Resumen del modelo
model_X.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        2080      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1568)              0         
                                                                 
 dropout (Dropout)           (None, 1568)              0

In [5]:
epochs = 100
batch_size = 32

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

history = model_X.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,
                    validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb])

Epoch 1/100
1719/1719 [==============================] - 23s 6ms/step - loss: 0.5762 - accuracy: 0.7882 - val_loss: 0.3588 - val_accuracy: 0.8738
Epoch 2/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3687 - accuracy: 0.8653 - val_loss: 0.2940 - val_accuracy: 0.8960
Epoch 3/100
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3226 - accuracy: 0.8833 - val_loss: 0.2881 - val_accuracy: 0.8948
Epoch 4/100
1719/1719 [==============================] - 10s 6ms/step - loss: 0.2940 - accuracy: 0.8927 - val_loss: 0.2534 - val_accuracy: 0.9082
Epoch 5/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2754 - accuracy: 0.8998 - val_loss: 0.2436 - val_accuracy: 0.9116
Epoch 6/100
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2589 - accuracy: 0.9055 - val_loss: 0.2402 - val_accuracy: 0.9116
Epoch 7/100
1719/1719 [==============================] - 9s 5ms/step - loss: 0.2468 - accuracy: 0.9096 - val_loss: 0.2318 - val_

In [6]:
accuracy = model_X.evaluate(X_valid, y_valid)[1]

print("Accuracy en Validación: ", accuracy)

157/157 [==============================] - 0s 3ms/step - loss: 0.2119 - accuracy: 0.9240
Accuracy en Validación:  0.9240000247955322


Me conformo con esto. Agarro algunos modelos que mandaron al discord.

In [7]:
#solución de Ariel O.

input_ = keras.layers.Input(shape=(X_train.shape[1:])) #Define el Layer de Inputs

conv1 = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(input_)
conv2 = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu')(conv1)

pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
#se pueden poner capas Dropout
dropout2 = keras.layers.Dropout(0.2)(pool2)

conv3 = keras.layers.Conv2D(filters=172, kernel_size=(3, 3), padding='same', activation='relu')(dropout2)
conv4 = keras.layers.Conv2D(filters=172, kernel_size=(3, 3), padding='valid', activation='relu')(conv3)

pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
#se pueden poner capas Dropout
dropout3 = keras.layers.Dropout(0.2)(pool3)

#defino entrada a una red ANN
flatten = keras.layers.Flatten()(dropout3)
hidden1 = keras.layers.Dense(256, activation="relu")(flatten)
dropout4 = keras.layers.Dropout(0.5)(hidden1)
hidden2 = keras.layers.Dense(128, activation="relu")(dropout4)
dropout5 = keras.layers.Dropout(0.2)(hidden2)
output = keras.layers.Dense(10, activation="softmax")(dropout5) # 10=cada clase resultado

model_ariel = keras.models.Model(inputs=[input_], outputs=[output])


model_ariel.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])

#No especifica estos valores, uso los que venia usando
epochs = 100
batch_size = 32

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

history = model_ariel.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,
                    validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb])


accuracy = model_ariel.evaluate(X_valid, y_valid)[1]

print("Accuracy en Validación: ", accuracy)



Epoch 1/100
1719/1719 [==============================] - 17s 8ms/step - loss: 0.5483 - accuracy: 0.8005 - val_loss: 0.3107 - val_accuracy: 0.8834
Epoch 2/100
1719/1719 [==============================] - 14s 8ms/step - loss: 0.3355 - accuracy: 0.8796 - val_loss: 0.2707 - val_accuracy: 0.8984
Epoch 3/100
1719/1719 [==============================] - 14s 8ms/step - loss: 0.2861 - accuracy: 0.8980 - val_loss: 0.2472 - val_accuracy: 0.9116
Epoch 4/100
1719/1719 [==============================] - 14s 8ms/step - loss: 0.2607 - accuracy: 0.9054 - val_loss: 0.2211 - val_accuracy: 0.9220
Epoch 5/100
1719/1719 [==============================] - 14s 8ms/step - loss: 0.2367 - accuracy: 0.9147 - val_loss: 0.2104 - val_accuracy: 0.9252
Epoch 6/100
1719/1719 [==============================] - 17s 10ms/step - loss: 0.2222 - accuracy: 0.9195 - val_loss: 0.2084 - val_accuracy: 0.9248
Epoch 7/100
1719/1719 [==============================] - 15s 8ms/step - loss: 0.2128 - accuracy: 0.9225 - val_loss: 0.1902 

Bueno, viene ganando Ariel!, a ver otra

In [8]:
#solución de Ezequiel92

input_ = keras.layers.Input(shape=(X_train.shape[1:]))

conv1 = keras.layers.Conv2D(filters=64, kernel_size=(4, 4), padding='same', activation='relu')(input_)
batch1 = keras.layers.BatchNormalization()(conv1)
conv2 = keras.layers.Conv2D(filters=64, kernel_size=(3, 3),  padding='valid',activation='relu')(batch1)

pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
dropout1 = keras.layers.Dropout(0.25)(pool1)

flatten = keras.layers.Flatten()(dropout1)
hidden1 = keras.layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(flatten)

dropout2 = keras.layers.Dropout(0.5)(hidden1)
output = keras.layers.Dense(10, activation="softmax")(dropout2)

model6_ezequiel = keras.models.Model(inputs=[input_], outputs=[output])

model6_ezequiel.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])


#No especifica estos valores, uso los que venia usando
epochs = 100
batch_size = 32

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

history = model6_ezequiel.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,
                    validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb])


accuracy = model6_ezequiel.evaluate(X_valid, y_valid)[1]

print("Accuracy en Validación: ", accuracy)



Epoch 1/100
1719/1719 [==============================] - 15s 8ms/step - loss: 0.7508 - accuracy: 0.8333 - val_loss: 0.5420 - val_accuracy: 0.8942
Epoch 2/100
1719/1719 [==============================] - 12s 7ms/step - loss: 0.6109 - accuracy: 0.8776 - val_loss: 0.5130 - val_accuracy: 0.9092
Epoch 3/100
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5893 - accuracy: 0.8863 - val_loss: 0.5368 - val_accuracy: 0.9042
Epoch 4/100
1719/1719 [==============================] - 11s 7ms/step - loss: 0.5684 - accuracy: 0.8927 - val_loss: 0.4966 - val_accuracy: 0.9152
Epoch 5/100
1719/1719 [==============================] - 11s 7ms/step - loss: 0.5565 - accuracy: 0.8958 - val_loss: 0.5030 - val_accuracy: 0.9160
Epoch 6/100
1719/1719 [==============================] - 11s 7ms/step - loss: 0.5441 - accuracy: 0.8984 - val_loss: 0.5073 - val_accuracy: 0.9144
Epoch 7/100
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5401 - accuracy: 0.8999 - val_loss: 0.4762 -

Sigue arriba Ariel, siguiente!

In [9]:
#solución de Claubress

model_Claubress = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(X_train.shape[1:])),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation="softmax")
])


model_Claubress.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])

#No especifica estos valores, uso los que venia usando
epochs = 100
batch_size = 32

early_stopping_cb=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

history = model_Claubress.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,
                    validation_data=(X_valid, y_valid),callbacks=[early_stopping_cb])


accuracy = model_Claubress.evaluate(X_valid, y_valid)[1]

print("Accuracy en Validación: ", accuracy)



Epoch 1/100
1719/1719 [==============================] - 10s 5ms/step - loss: 0.5015 - accuracy: 0.8129 - val_loss: 0.3061 - val_accuracy: 0.8882
Epoch 2/100
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3488 - accuracy: 0.8706 - val_loss: 0.2803 - val_accuracy: 0.8958
Epoch 3/100
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3112 - accuracy: 0.8850 - val_loss: 0.2476 - val_accuracy: 0.9058
Epoch 4/100
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2883 - accuracy: 0.8928 - val_loss: 0.2288 - val_accuracy: 0.9118
Epoch 5/100
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2716 - accuracy: 0.8987 - val_loss: 0.2267 - val_accuracy: 0.9140
Epoch 6/100
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2616 - accuracy: 0.9022 - val_loss: 0.2190 - val_accuracy: 0.9202
Epoch 7/100
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2479 - accuracy: 0.9082 - val_loss: 0.2143 - val_

In [10]:
'''
Y SOBRE EL FINAL DE LA RECTA GANÓ CLAUDIO!!!

Veamos como le da en test y esa es la solución

Nota: En realidad si se quiere hacer una comparación mejor habría que correr varias veces, sacar un promedio, hacer un análisis más profundo, etc.
Pero como primera aproximación está más que bien, vean el detalle del link de kaggle para tener idea
'''

accuracy_test = model_Claubress.evaluate(X_test, y_test)[1]
print("Solución de AP:", accuracy_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.2113 - accuracy: 0.9243
Solución de AP: 0.9243000149726868


El objetivo era ganarle a la totalmente conectada y eso se cumplió (en las soluciones de redes totalmente conectadas para Fashion-MNIST de la semana anterior nunca se pudo pasar del ochenta y algo) el enunciado tambien mencionaba PCA, eso se me pasó y fue por pensar en las totalmente conectadas, pero no tiene sentido para CNN (o sea, se puede usar y pasarle una Conv1D, pero se pierde el chiste de una CNN).

Me puse a buscar que onda LA MEJOR solución para este dataset y parece que la verdad que estamos todos ahí (no es como MNIST que con CNN se alcanza como 0.99 de precisión).

Si quieren ver lo mejor que consiguieron en [KAGGLE](https://www.kaggle.com/code/pankssid/fashion-mnist-solution), interesante ver el análisis por clase y ver como algunas son más dificiles que otras.

Extra: Si quieren ver una búsqueda de arquitectura realmente a conciencia vean a [este](https://www.kaggle.com/code/cdeotte/how-to-choose-cnn-architecture-mnist/notebook) enfermo sacarle hasta el último bit de jugo a MNIST.

---

Como comentario para los que mandaron una solución (y los demás ténganlo en cuenta también), recuerden que **NO ALCANZA** solo con la arquitectura, tienen que decir que optimizador usaron, si pusieron callback de early_stopping, el tamaño del batch, etcetc.

---



